In [5]:
import polars as pl

In [197]:
df = pl.read_parquet('data/cnb_vyber.parquet')

In [9]:
import re
import statistics
from src.najdi_rok import najdi_rok

In [199]:
df = df.with_columns(pl.col('008').map_elements(najdi_rok).alias('rok').cast(int))

C:\Users\micha\AppData\Local\Temp\ipykernel_11840\4262393550.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = df.with_columns(pl.col('008').map_elements(najdi_rok).alias('rok').cast(int))


In [201]:
petdvanula = pl.read_parquet("data/cnb_sloupce/520.parquet")

In [203]:
df = df.join(petdvanula, on='001', how='left')

In [207]:
tematicke_sloupce = ['072_x','245_a','245_p','246_a','500_a','520_a','520_b','521_a','648_a','650_a','650_x','650_z','650_y','651_a','653_a','655_a','964_a']

In [209]:
df = df.select(pl.col(['rok'] + tematicke_sloupce)).with_columns(pl.col(tematicke_sloupce).cast(pl.List(pl.String)).list.join(" "))
df = df.with_columns(pl.col(tematicke_sloupce).fill_null(' ')).with_columns(pl.concat_str([pl.col(t) for t in tematicke_sloupce], separator=" ").alias("keywords"))

In [373]:
def tema(keyw):
    return df.filter(pl.col('rok') >= 1945).filter(pl.col('keywords').str.contains(f'(?i){keyw}')).with_columns(tema = pl.lit(keyw))

In [397]:
tema1 = tema('fotbal')
tema2 = tema('hokej')
tema3 = tema('florbal')
tema4 = tema('gorodky')
tema5 = tema('šachy')
tema6 = tema('baseball')
tema7 = tema('branná')
tema8 = tema('formule 1')
do_grafu = pl.concat([tema1, tema2, tema3, tema4, tema5, tema6, tema7, tema8])

In [239]:
df.drop_nulls(subset='rok').filter(pl.col('keywords').str.contains('kyberneti')).group_by('rok').len().sort('rok')

rok,len
i64,u32
1956,2
1957,4
1958,2
1959,1
1960,7
…,…
2020,10
2021,15
2022,11


In [229]:
import altair as alt

In [441]:
def irozhlas():

    return {
        "config" : {
             "title": {'font': "Noticia Text",
                      'fontSize': 20,
                      'anchor': 'start',
                    'subtitleFont': 'Noticia Text',
                      'subtitleFontSize': 14,
                      'subtitlePadding': 14,
                       'dy': -14
                      },
             "axis": {
                "labelFont": "Asap",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 11,
                "titleFontSize": 14,
                'labelPadding': 2,
                 'titlePadding': 10
            },
            "legend": {
                "labelFont": "Asap",
                "labelFontWeight": "normal",
                "titleFont": "Asap",
                "titleFontWeight": "normal",
                "labelFontSize": 11,
                "titleFontSize": 14
            },
        }
    }

alt.themes.register('irozhlas', irozhlas)
alt.themes.enable('irozhlas')

RendererRegistry.enable('default')

In [431]:
import json

TypeError: the JSON object must be str, bytes or bytearray, not NumberLocale

In [447]:
gaussian_jitter = alt.Chart(do_grafu, title='Normally distributed jitter').mark_circle(size=8).encode(
    x=alt.X("rok:O", title=None),
y=alt.Y("tema:N", title=None),
   yOffset=alt.YOffset("jitter:Q", scale=alt.Scale(range=[0, 35])),
    color=alt.Color('tema:N').legend(None)
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter="sqrt(-2*log(random()))*cos(2*PI*random())"
)

gaussian_jitter.properties(
    width=400,
    height=350
)

alt.Chart(...)